# Edits saved without JS enabled

[Task](https://phabricator.wikimedia.org/T240697)

# Purpose

We need to determine the percentage of completed (non-bot) edits without javascript (JS) support (either in a browser that doesn't have JS support, or a regular browser with JS turned off). This data will be used to ensure that we are accurately accounting for users that complete edits with and without JS support. 

# Metrics

* Snapshot % of user edits done with no JS support, browseable by project and platform (mobile vs. desktop)
* Snapshot % of anon edits done with no JS support, browseable by project and platform (mobile vs. desktop)
* Snapshot % of all edits done with no JS support, browseable by project and platform (mobile vs. desktop)

# Methodology

This analysis builds on the work completed in [T240697#6217971](https://phabricator.wikimedia.org/T240697#6217971). 

In this first analysis, we were unable to determine an accurate number of completed Wikieditor edits by users without JS due to ad and tracking blockers that block sending data via EventLogging. Instrumentation was added in [T263505](https://phabricator.wikimedia.org/T263505) to send a new event when an edit made with Wikieditor is saved by a user with JS enabled. We started recording these events on 12 May 2021 following the deployment of a fix to the instrumentation.

The following event was added to the VisualEditorFeatureUse eventlogging schema to track these events.

`event.feature = 'mwSave'`
`event.action = 'source-has-js'`

This instrumentation only applies to saved edits made with WikiEditor. In this analysis, we assume all non-wikitext edits (`editor_interface ! = 'wikitext'`) or non-page edits (`integration ! = 'page'`)(i.e. edits made with VisualEditor, Discussion Tool or app edits) have JS enabled as these editors require JS to work. 



# Data Clarifications and Assumptions

* Data reflects events logged from 13 May 2021 through 29 May 2021.
* Analysis based on data logged in EditAttemptStemp and VisualEditorFeatureUse, which has a sampling rate of 6.25%
* Assumes all non-Wikieditor page edits (such as, Visual Editor and Discussion Tool edits) were completed with JS support, as these editors require JS.
* Reviewed data excludes identified bots; however, unidentified automated traffic may be included.
* Instrumentation does differentiate between different possible reasons JS was not available at the time of the save, such as users that:
  * intentionally disabled JS for privacy reasons,
  * were not able to load JS prior to saving,
  * have browsers that do not support JS,
  * has an older version of JS we don't send JS to, or
  * users that disabled the 2010 Wikieditor toolbar by user preference. Note these users technically had JS at the time of the save; however, current instrumentation does not log their support 

In [34]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); 
    library(lubridate); 
    library(scales);
    library(magrittr); 
    library(dplyr);
})


# Percent of edits completed without JS by platform

In [433]:
query <-
"
-- We are only interested in saveSuccess events i.e. saved edits -- 
WITH saveSuccess_sessions as (
    SELECT DISTINCT
      event.editing_session_id AS ss_session_id,
      event.user_id AS user_id,
      event.editor_interface AS editor_interface,
      event.platform AS platform,
      event.integration AS integration,
      useragent.device_family as user_agent,
      wiki AS ss_wiki
FROM event.editattemptstep
WHERE
    YEAR = 2021
    AND MONTH = 05
-- May 13 was the first full day of events
    AND (Day >= 13 AND DAY <= 29)
    AND event.action = 'saveSuccess' 
    AND useragent.is_bot = false
-- remove any oversampled events
   AND NOT event.is_oversample
),

-- Now we will use 'source-has-js' events to identify wikitext edits saved with JS support 
js_wt_sessions AS(
  SELECT
    event.editingSessionid AS wt_js_session_id,
    wiki AS js_wt_wiki,
    event.action AS wikitext_js_edit
FROM event.visualeditorfeatureuse 
WHERE
    YEAR = 2021
    AND MONTH = 05
    AND (DAY >= 13 AND DAY <= 29)
    AND event.action = 'source-has-js'
    AND event.feature = 'mwSave'
-- only wikitext events should have this action recorded
    AND event.editor_interface = 'wikitext'
    AND useragent.is_bot = false
)


-- MAIN QUERY 
SELECT 
  wiki,
  user_agent,
-- only wikitext page edits without a js event should be considered as not having js
-- WikiEditor records all edits as desktop due to bug See T249944
  SUM(CAST(user_id = 0 AND editor_interface = 'wikitext' AND integration = 'page' AND platform = 'desktop' AND wikitext_js_edit is NULL AS INT)) AS anon_no_js_edit,
  SUM(CAST(user_id != 0 AND editor_interface = 'wikitext' AND integration = 'page' AND platform = 'desktop' AND wikitext_js_edit is NULL AS INT)) AS user_no_js_edit,
  SUM(CAST(user_id = 0 AS INT)) AS anon_all_edits,
  SUM(CAST(user_id != 0 AS INT)) AS user_all_edits 
FROM (
  SELECT 
    user_id AS user_id ,
    ss_wiki AS wiki,
    editor_interface AS editor_interface,
    integration AS integration,
    platform AS platform,
    user_agent AS user_agent,
    ss_session_id AS ss_session_id,
    wikitext_js_edit
  FROM saveSuccess_sessions ss  
  LEFT JOIN 
    js_wt_sessions ON ss.ss_session_id = js_wt_sessions.wt_js_session_id
    AND ss.ss_wiki = js_wt_sessions.js_wt_wiki
  GROUP BY 
    ss.user_id, 
    ss.ss_wiki,
    ss.editor_interface,
    ss.integration,
    ss.platform,
    ss.user_agent,
    ss.ss_session_id, 
    js_wt_sessions.wikitext_js_edit
    ) edit_sessions
GROUP BY
    wiki,
    user_agent
  "

In [434]:
collect_js_edits <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



## Identify Mobile Vs Desktop edits based on device family

WikiEditor is only capable of recording edits as platform = "desktop" (rather than "tablet" or "phone"). As a result, we used the device family recorded for each edit to detect mobile and desktop edits.

In [435]:
patterns <-  c('mobi','240x240','240x320','320x320','alcatel','android','audiovox','bada','benq','blackberry','cdm-',
            'compal-','docomo','ericsson','hiptop','htc[-_]','huawei','ipod','kddi-','kindle', 'meego','midp','mitsu',
            'mmp\\/','mot-','motor','ngm_','nintendo','opera.m','palm','panasonic','philips','phone','playstation',
            'portalmmm','sagem-','samsung','sanyo','sec-','sendo','sharp','silk','softbank','symbian','teleca',
             'up.browser','webos')

In [436]:
patterns_start <- c('^lg-', '^sie-', '^nec-', '^lge-', '^sgh-', '^pg-')

In [437]:
js_edits_wplatform <- collect_js_edits %>%
    mutate(platform = ifelse((grepl(paste(patterns, collapse = "|"), user_agent, ignore.case = TRUE) |
                            grepl(paste(patterns_start, collapse = "|"), user_agent, ignore.case = TRUE)) , 
                             'mobile', 'desktop'))


In [438]:
write.csv(js_edits_wplatform, "js_edits_wplatform.csv", row.names=FALSE)

## Snapshot % of logged-in user edits done with no JS support, by platform (mobile vs. desktop)

In [439]:
nojs_user_edits <- js_edits_wplatform %>%
    group_by(platform) %>%
    summarize(pct_nonjs_edit = paste0(round(sum(user_no_js_edit)/sum(user_all_edits) * 100,2), "%"))

nojs_user_edits 

`summarise()` ungrouping output (override with `.groups` argument)



platform,pct_nonjs_edit
<chr>,<chr>
desktop,6.1%
mobile,3.15%


## Snapshot % of anon edits done with no JS support, by platform (mobile vs. desktop)

In [428]:
nojs_anon_edits <- js_edits_wplatform %>%
    #filter(logged_in_status == 'logged_out')  %>%
    group_by(platform) %>%
    summarize(pct_nonjs_edit = paste0(round(sum(anon_no_js_edit)/sum(anon_all_edits) * 100,2), "%"))

nojs_anon_edits 

`summarise()` ungrouping output (override with `.groups` argument)



platform,pct_nonjs_edit
<chr>,<chr>
desktop,0.98%
mobile,1.57%


## Snapshot % of all edits done with no JS support, browseable by project and platform (mobile vs. desktop)

In [440]:
nojs_all_edits <- js_edits_wplatform %>%
    group_by(platform) %>%
    summarize(pct_nonjs_edit = paste0(round((sum(user_no_js_edit) +sum(anon_no_js_edit))
                                            /(sum(user_all_edits) + sum(anon_all_edits)) * 100,2), "%"))

nojs_all_edits 

`summarise()` ungrouping output (override with `.groups` argument)



platform,pct_nonjs_edit
<chr>,<chr>
desktop,5.28%
mobile,2.49%


## By Project

Data will be added to Superset to easily browse by all projects.